In [2]:
import gluonts
import mxnet

print(gluonts.__version__)
print(mxnet.__version__)

0.4.1
1.4.1


In [3]:
# standard imports
import numpy as np
import pandas as pd

# Vis
%matplotlib inline
import matplotlib.pyplot as plt
from pprint import pprint

# mxnet
import mxnet as mx

# gluonts
from gluonts.dataset.util import to_pandas

# imports from utility function
import utils_gluonts10 as utils
from utils_gluonts10 import get_dataset, dataset_recipes

In [4]:
print(f"Available datasets: {list(dataset_recipes.keys())}")

Available datasets: ['m4_daily', 'm4_daily_domain', 'm4_hourly', 'm4_monthly', 'm4_monthly_atm', 'm4_monthly_dates', 'm4_monthly_demographic', 'm4_monthly_domain', 'm4_monthly_finance', 'm4_monthly_industry', 'm4_monthly_macro', 'm4_monthly_micro', 'm4_monthly_other', 'm4_quarterly', 'm4_quarterly_atm', 'm4_quarterly_dates', 'm4_quarterly_demographic', 'm4_quarterly_domain', 'm4_quarterly_finance', 'm4_quarterly_industry', 'm4_quarterly_macro', 'm4_quarterly_micro', 'm4_quarterly_other', 'm4_weekly', 'm4_weekly_domain', 'm4_weekly_id', 'my_weekly_tm', 'm4_yearly', 'm4_yearly_demographic', 'm4_yearly_domain', 'm4_yearly_finance', 'm4_yearly_industry', 'm4_yearly_macro', 'm4_yearly_micro', 'm4_yearly_other']


In [12]:
dataname = "m4_monthly"
dataset = utils.get_dataset(dataname, regenerate=False)

INFO:root:using dataset already processed in path C:\Users\TM\.mxnet\gluon-ts\datasets\m4_monthly.


In [6]:
dataname = "m4_weekly_domain"
dataset = utils.get_dataset(dataname, regenerate=False)

INFO:root:using dataset already processed in path C:\Users\TM\.mxnet\gluon-ts\datasets\m4_weekly_domain.


In [13]:
# get the first time series in the training set
train_entry = next(iter(dataset.train))
test_entry = next(iter(dataset.test))

# check keys
print(train_entry.keys())
print(test_entry.keys())

dict_keys(['start', 'target', 'feat_static_cat', 'source'])
dict_keys(['start', 'target', 'feat_static_cat', 'source'])


In [14]:
train_entry

{'start': Timestamp('1750-01-31 00:00:00', freq='M'),
 'target': array([ 8000.,  8350.,  8570.,  7700.,  7080.,  6520.,  6070.,  6650.,
         6830.,  5710.,  5260.,  5470.,  7870.,  7360.,  8470.,  7880.,
         6750.,  6860.,  6220.,  6650.,  5450.,  5280.,  4970.,  5550.,
         7750.,  7760.,  7190.,  7440.,  6590.,  6210.,  6010.,  6390.,
         5780.,  5700.,  4680.,  4970.,  6850.,  6740.,  7490.,  6250.,
         6900.,  5870.,  5610.,  6310.,  6110.,  6090.,  5810.,  6060.,
         6950.,  7820.,  8270.,  7840.,  7850.,  6710.,  6220.,  7430.,
         6560.,  7080.,  6680.,  6900.,  8680.,  8450.,  8670.,  8470.,
         7910.,  8140.,  7310.,  7860.,  7730.,  7330.,  7430.,  7150.,
         8720.,  8340.,  8840.,  8780.,  8250.,  8180.,  7570.,  9280.,
         9220.,  9140.,  8950.,  8540.,  9360.,  9750.,  9270.,  8690.,
         8200.,  7610.,  7160.,  8280.,  7370.,  7130.,  6840.,  7260.,
         7430.,  7950.,  7790.,  8090.,  7480.,  6700.,  6650.,  6960.,


In [15]:
dataset.metadata.feat_static_cat

[<CategoricalFeatureInfo name='feat_static_cat' cardinality='48000'>]

In [16]:
[feat_static_cat.cardinality for feat_static_cat in dataset.metadata.feat_static_cat]

['48000']

### Plot Series

In [17]:
test_series = to_pandas(test_entry)
train_series = to_pandas(train_entry)

fig, ax = plt.subplots(2, 1, sharex=True, sharey=True, figsize=(10, 7))

train_series.plot(ax=ax[0])
ax[0].grid(which="both")
ax[0].legend(["train series"], loc="upper left")

test_series.plot(ax=ax[1])
ax[1].axvline(train_series.index[-1], color='r') # end of train dataset
ax[1].grid(which="both")
ax[1].legend(["test series",c="upper left")
 "end of train series"], lo
plt.show()

SyntaxError: invalid syntax (<ipython-input-17-8a6d39593d1f>, line 13)

## DeepAR - with 2 static features

In [7]:
# gluonts
from gluonts.trainer import Trainer
from gluonts.evaluation.backtest import make_evaluation_predictions
from gluonts.evaluation import Evaluator

# DeepAREstimator
from gluonts.model.deepar import DeepAREstimator

In [8]:
######################
### deepar wrapper ###
def deepar(data="m4_quarterly", seed=42, epochs=100, batches=50):

    dataset = get_dataset(data, regenerate=False)
    mx.random.seed(seed)
    np.random.seed(seed)

    trainer = Trainer(
        ctx=mx.cpu(0),
        epochs=epochs,
        num_batches_per_epoch=batches,
        learning_rate=1e-3,
    )
    
    estimator = DeepAREstimator(
        trainer=trainer,
        cardinality= [
            feat_static_cat.cardinality 
            for feat_static_cat in dataset.metadata.feat_static_cat
        ],
        context_length=dataset.metadata.prediction_length,
        prediction_length=dataset.metadata.prediction_length,
        freq=dataset.metadata.freq,
        use_feat_static_cat=True
    )

    # predictor = estimator.train(training_data=dataset.train,
    #                             validation_data=dataset.test)
    predictor = estimator.train(training_data=dataset.train)

    forecast_it, ts_it = make_evaluation_predictions(
        dataset.test, predictor=predictor, num_samples=100
    )

    agg_metrics, item_metrics = Evaluator()(
        ts_it, forecast_it, num_series=len(dataset.test)
    )

    metrics = ["MASE", "sMAPE", "MSIS", "wQuantileLoss[0.5]", "wQuantileLoss[0.9]"]
    output = {key: round(value, 8) for key, value in agg_metrics.items() if key in metrics}

    output["seed"] = seed
    output["epochs"] = epochs
    output["num_batches"] = batches

    df = pd.DataFrame([output])

    return df


In [26]:
results = pd.DataFrame()

for i in range(100, 501, 50):
    print("Epochs:", i)
    df = deepar(data="m4_weekly", seed=42, epochs=i, batches=100)
    pprint(df)
    results = results.append(df)

results

INFO:root:downloading and processing m4_weekly


Epochs: 100
saving time-series into /home/ec2-user/.mxnet/gluon-ts/datasets/m4_weekly/train/data.json


INFO:root:Start model training
INFO:root:Epoch[0] Learning rate is 0.001
  0%|          | 0/100 [00:00<?, ?it/s]

saving time-series into /home/ec2-user/.mxnet/gluon-ts/datasets/m4_weekly/test/data.json


INFO:root:Number of parameters in DeepARTrainingNetwork: 49593
100%|██████████| 100/100 [00:03<00:00, 25.30it/s, avg_epoch_loss=7.33]
INFO:root:Epoch[0] Elapsed time 3.955 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=7.333318
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 100/100 [00:03<00:00, 26.80it/s, avg_epoch_loss=6.83]
INFO:root:Epoch[1] Elapsed time 3.733 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=6.825292
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 100/100 [00:03<00:00, 26.90it/s, avg_epoch_loss=6.71]
INFO:root:Epoch[2] Elapsed time 3.720 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=6.714796
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 100/100 [00:03<00:00, 27.30it/s, avg_epoch_loss=6.53]
INFO:root:Epoch[3] Elapsed time 3.665 seconds
INFO:root:Epoch[3] Evaluation metric 'epoch_loss'=6.526230
INFO:root:Epoch[4] Learning rate is 0.001
100%|██████████| 100/100 [00:03<00:00, 26.82it/s, avg_epoch_loss=

100%|██████████| 100/100 [00:03<00:00, 27.24it/s, avg_epoch_loss=5.84]
INFO:root:Epoch[37] Elapsed time 3.674 seconds
INFO:root:Epoch[37] Evaluation metric 'epoch_loss'=5.839576
INFO:root:Epoch[38] Learning rate is 0.001
100%|██████████| 100/100 [00:03<00:00, 27.23it/s, avg_epoch_loss=5.88]
INFO:root:Epoch[38] Elapsed time 3.673 seconds
INFO:root:Epoch[38] Evaluation metric 'epoch_loss'=5.882671
INFO:root:Epoch[39] Learning rate is 0.001
100%|██████████| 100/100 [00:03<00:00, 26.38it/s, avg_epoch_loss=5.85]
INFO:root:Epoch[39] Elapsed time 3.792 seconds
INFO:root:Epoch[39] Evaluation metric 'epoch_loss'=5.848744
INFO:root:Epoch[40] Learning rate is 0.001
100%|██████████| 100/100 [00:03<00:00, 27.45it/s, avg_epoch_loss=5.82]
INFO:root:Epoch[40] Elapsed time 3.647 seconds
INFO:root:Epoch[40] Evaluation metric 'epoch_loss'=5.820178
INFO:root:Epoch[41] Learning rate is 0.001
100%|██████████| 100/100 [00:03<00:00, 26.68it/s, avg_epoch_loss=5.84]
INFO:root:Epoch[41] Elapsed time 3.750 second

INFO:root:Epoch[74] Learning rate is 0.0005
100%|██████████| 100/100 [00:03<00:00, 27.66it/s, avg_epoch_loss=5.71]
INFO:root:Epoch[74] Elapsed time 3.617 seconds
INFO:root:Epoch[74] Evaluation metric 'epoch_loss'=5.712237
INFO:root:Epoch[75] Learning rate is 0.0005
100%|██████████| 100/100 [00:03<00:00, 27.43it/s, avg_epoch_loss=5.68]
INFO:root:Epoch[75] Elapsed time 3.647 seconds
INFO:root:Epoch[75] Evaluation metric 'epoch_loss'=5.683760
INFO:root:Epoch[76] Learning rate is 0.0005
100%|██████████| 100/100 [00:03<00:00, 27.04it/s, avg_epoch_loss=5.7]
INFO:root:Epoch[76] Elapsed time 3.701 seconds
INFO:root:Epoch[76] Evaluation metric 'epoch_loss'=5.701236
INFO:root:Epoch[77] Learning rate is 0.0005
100%|██████████| 100/100 [00:03<00:00, 27.39it/s, avg_epoch_loss=5.7]
INFO:root:Epoch[77] Elapsed time 3.655 seconds
INFO:root:Epoch[77] Evaluation metric 'epoch_loss'=5.699781
INFO:root:Epoch[78] Learning rate is 0.0005
100%|██████████| 100/100 [00:03<00:00, 27.11it/s, avg_epoch_loss=5.74]

KeyboardInterrupt: 

### m4_daily vs. m4_daily domain

- reload `utils`
- test `m4_daily` vs. `m4_daily_domain`
- ten trials each, 
 - `epochs=100`, `num_batches_per_epoch=100`

...

## `m4_monthly_domain` vs. `m4_monthly`

...

## `m4_quarterly` vs. `m4_quarterly_domain`

...

## `m4_yearly` vs. `m4_yearly_domain`

...

## `m4_quarterly` vs `m4_quarterly_domain`

In [ ]:
results = pd.DataFrame()

for i in range(42, 52):
    print("Seed:", i)
    df = deepar(data="m4_yearly", seed=i, epochs=100, batches=100)
    pprint(df)
    results = results.append(df)

results

## m4_hourly

In [ ]:
results = pd.DataFrame()

for i in range(42, 52):
    print("Seed:", i)
    df = deepar(data="m4_hourly", seed=i, epochs=100, batches=100)
    pprint(df)
    results = results.append(df)

results

In [ ]:
results = pd.DataFrame()

for i in range(42, 52):
    print("Seed:", i)
    df = deepar(data="m4_hourly", seed=i, epochs=200, batches=100)
    pprint(df)
    results = results.append(df)

results

In [ ]:
!jupyter nbconvert --output-dir="../html_outputs" --to html gluonts-use_multiple_static_feats.ipynb